### Decision Tree Implementation

In [5]:
import pandas as pd
import numpy as np
#Plotly = "fancy" Matplotlib/Seaborn
import plotly.graph_objects as go

In [6]:
# read the csv file
df = pd.read_csv("fish.csv") 

# how many rows and columns?
print(df.shape)

# print column names
print(df.columns)

# print class distribution
print(df["type"].value_counts())

(1000, 3)
Index(['length', 'weight', 'type'], dtype='object')
type
tuna      608
salmon    392
Name: count, dtype: int64


In [7]:
# use the following function on the "type" column with the variable x assigned to the value of the "type" column: change it to 1 if tuna else change it to 0
df["type"] = df["type"].apply(lambda x: 1 if x=="tuna" else 0)

In [8]:
# create a Figure
fig = go.Figure()

# specify custom colors for the plot
color_map = {
    0: "red",
    1: "blue",
}

# apply the color map to 'type' column
colors = df["type"].map(color_map)

# add a scatter trace to the figure
fig.add_trace(go.Scatter(x=df["length"], 
            y=df["weight"],
            mode="markers",
            marker=dict(color=colors, size=8)))

# add x-label, y-label and title
fig.update_layout(
    width=800,
    height=600,  
    title_text="Scatter Plot of Data",
    xaxis=dict(title="length",
               tickvals=[i for i in range(10)]),
    yaxis=dict(title="weight",
               tickvals=[i for i in range(10)])
)


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': array(['blue', 'blue', 'blue', ..., 'blue', 'blue', 'blue'], dtype=object),
                         'size': 8},
              'mode': 'markers',
              'type': 'scatter',
              'x': array([8.61838093, 2.52204585, 2.0356904 , ..., 2.4100875 , 0.990773  ,
                          7.51977792]),
              'y': array([7.84803016, 8.76250711, 7.10197149, ..., 3.3557809 , 3.81221028,
                          5.39281728])}],
    'layout': {'height': 600,
               'template': '...',
               'title': {'text': 'Scatter Plot of Data'},
               'width': 800,
               'xaxis': {'tickvals': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'title': {'text': 'length'}},
               'yaxis': {'tickvals': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'title': {'text': 'weight'}}}
})

In [ ]:
features = ["length", "weight"]

In [ ]:
def compute_entropy(vals):
    """
    Input:
    vals: list of 0s and 1s corresponding to two classes
    
    Output:
    entropy: float"""

    # probability of class labeled as 1 
    # will be equal to the average of vals
    p1 = np.mean(vals)
    p0 = (1-p1)

    if p1==0 or p0==0: # it means data is homoegeneous
        return 0

    return - (p0*np.log2(p0) + p1*np.log2(p1)) # formula of entropy for two classes

In [ ]:
def compute_gini(vals):
    """
    Input: vals is a list of 0s and 1s
    Output:
    gini: float
    """

    # probability of '1' will be equal to the average of vals
    p1 = np.mean(vals)
    p0 = (1-p1) # since there are just two classes and p0+p1 = 1

    if p1==0 or p0==0:
        return 0

    return 1 - p1**2 - p0**2

In [ ]:
def compute_impurity(df, feature, val, criterion):
    """
    Inputs:
    df: dataframe before splitting
    feature: colname to test for best split
    val: value of 'feature' to test for best split

    Output: float
    Returns the entropy after split
    """

    # Make the split at (feature, val)
    left = df[df[feature]<=val]["type"]
    right = df[df[feature]>val]["type"]

    # calculate impurity of both partitions

    if criterion=="entropy":
        left_impurity = compute_entropy(left)
        right_impurity = compute_entropy(right)
    else:
        left_impurity = compute_gini(left)
        right_impurity = compute_gini(right)


    # return weighted entropy
    n = len(df) # total number of data points
    left_n = len(left) # number of data points in left partition
    right_n = len(right) # number of data points in right partition

    return (left_n/n)*left_impurity + (right_n/n)*right_impurity


In [ ]:
# Finding the first split:

# initialize best_params which is a dictionary that will keep track
# of best feature and split value at each node.
best_params = {"feature": None, "impurity": np.inf, "split_value": None}

# for each feature in features, do the following:
### for val in all feature values 
### (starting from the min possible value of the feature until max possible value, 
### incrementing by 'step_size'), check the following:
###### if impurity at this feature val is less than previously recorded impurity, 
###### then update best_params

# Following is the code for above interpretation
for feature in features:
    curr_val = df[feature].min()
    step_size = 0.1
    while curr_val <= df[feature].max():
        curr_feature_split_impurity = compute_impurity(df, feature, curr_val, "entropy")
        if curr_feature_split_impurity < best_params["impurity"]:
            best_params["impurity"] = curr_feature_split_impurity
            best_params["feature"] = feature
            best_params["split_value"] = curr_val
        curr_val += step_size


best_params

{'feature': 'length',
 'impurity': 0.6843553171032571,
 'split_value': 3.0065456626365483}

In [ ]:
print(best_params)

{'feature': 'length', 'impurity': 0.6843553171032571, 'split_value': 3.0065456626365483}


In [ ]:
def get_best_params(df, features, criterion):
    """
    A function to determine the best split at a node
    
    Input:
    df: dataframe before split
    features: list of features
    criterion: impurity measure to use (gini or entropy)
    
    Output: 
    best_params: dict
    """
    
    # Initialize best_params
    best_params = {"feature": None, "val": None, "impurity": np.inf}


    # iterate for all features
    for feature in features:
        curr_val = df[feature].min()
        step_size = 0.1
        # iterate for all values for a feature (according to step_size)
        while curr_val<=df[feature].max():
            # calculate impurity (gini or entropy) for the current value of feature
            impurity = compute_impurity(df, feature, curr_val, criterion)

            # update best_params if impurity is less than previous impurity
            if impurity <= best_params["impurity"]:
                best_params["feature"] = feature
                best_params["val"] = curr_val
                best_params["impurity"] = impurity
            curr_val += step_size
    
    best_params["val"] = np.round(best_params["val"], 2)
    best_params["impurity"] = np.round(best_params["impurity"], 2)
    
    return best_params

In [ ]:
def build_tree(data, features, curr_depth=0, max_depth=3, criterion="entropy"):
    """A function to build the decision tree recursively.
    
    Input:
    data: dataframe with columns length, weight, type
    features: ['length', 'weight']
    curr_depth: Keep track of depth at current node
    max_depth: Decision tree will stop growing if max_depth reached
    criterion: "gini" or "entropy" 

    """
    
    # Base case: max depth reached or no features left
    if curr_depth >= max_depth or not features:
        classes, counts = np.unique(data['type'].values, return_counts=True)
        predicted_class = classes[np.argmax(counts)]
        print(("--" * curr_depth) + f"Predict: {predicted_class}")
        return

    # Get the best feature and value to split the data
    best_params = get_best_params(data, features, criterion)
    
    # Base case: pure node
    if best_params["impurity"] == 0:
        predicted_class = data['type'].iloc[0]
        print(("--" * curr_depth) + f"Predict: {predicted_class}")
        return
    
    # If there's no feature that can improve the purity (not possible to split)
    if best_params["feature"] is None:
        predicted_class = data['type'].iloc[0]
        print(("--" * curr_depth) + f"Predict: {predicted_class}")
        return

    # Print the current question (decision rule)
    best_feature = best_params["feature"]
    best_split_val = best_params["val"]
    question = f"Is {best_feature} <= {best_split_val}?"
    print(("--" * (curr_depth*2)) + ">" + f"{question}")
    
    # Split the dataset
    left_df = data[data[best_feature] <= best_split_val]
    right_df = data[data[best_feature] > best_split_val]

    # Recursive calls for left and right subtrees
    if not left_df.empty:
        print(("--" * curr_depth) + f"Yes ->")
        build_tree(left_df, features, curr_depth + 1, max_depth, criterion)
    
    if not right_df.empty:
        print(("--" * curr_depth) + f"No ->")
        build_tree(right_df, features, curr_depth + 1, max_depth, criterion)

In [ ]:
build_tree(df, ["length", "weight"], max_depth=4, criterion="entropy")

>Is length <= 3.01?
Yes ->
--Predict: 1
No ->
---->Is weight <= 4.0?
--Yes ->
----Predict: 0
--No ->
-------->Is length <= 6.98?
----Yes ->
------------>Is length <= 3.38?
------Yes ->
--------Predict: 0
------No ->
--------Predict: 1
----No ->
------Predict: 1


|Tree|Visual|
|-----|-----|
|![image.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*CzRPaOHVNxLl64ph8__g8A.png)|![image.png](https://miro.medium.com/v2/resize:fit:828/format:webp/1*6_K97AKzkotw1a7XqcvJjw.png)|

